In [1]:
import pandas as pd
import requests
import os, sys
from fuzzywuzzy import fuzz
import dedupe
import numpy as np
import shutil

/home/andrew/miniconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [108]:
#Get TV Stations by DMA
dma = 'PHOENIX (PRESCOTT)'
url = "https://publicfiles.fcc.gov/api/service/facility/search/"+dma.split()[0]+".json"
r = requests.get(url).json()
tv = pd.DataFrame(r['results']['globalSearchResults']['tvFacilityList'])
tv = tv[tv.nielsenDma==dma]
tv = tv[['callSign','communityState','networkAfil','nielsenDma','virtualChannel','id']]
tv['medium'] = "broadcast"
tv = tv.rename(columns={'callSign':'sign', 'communityState':'state', 'networkAfil': 'network',
                       'nielsenDma':'dma', 'virtualChannel':'channel'})
tv.to_csv('stationsPhoenix.csv', index=False)

,callSign,communityState,networkAfil,nielsenDma,virtualChannel,id,medium
10,KTVK,AZ,CW,PHOENIX (PRESCOTT),3,40993,Broadcast


In [103]:
print(r)

{'status': 'OK', 'responseTime': 468, 'message': '103 Facilities Found', 'results': {'globalSearchResults': {'tvResultsCount': 20, 'fmResultsCount': 14, 'amResultsCount': 14, 'cableResultsCount': 55, 'cableZipCodeResultsCount': 0, 'sdarsResultsCount': 0, 'dbsResultsCount': 0, 'tvFacilityList': [{'id': '2728', 'callSign': 'KAET', 'service': 'Digital TV', 'rfChannel': '8', 'virtualChannel': '8', 'licenseExpirationDate': '10/01/2022', 'statusDate': '05/14/2019', 'status': 'LICENSED', 'communityCity': 'PHOENIX', 'communityState': 'AZ', 'facilityType': 'EDT', 'frequency': '180.0', 'activeInd': 'Y', 'scannedLetterIds': '', 'partyName': 'ARIZONA BOARD OF REGENTS FOR ARIZONA STATE UNIVERSITY', 'partyAddress1': '555 N. CENTRAL AVENUE', 'partyAddress2': '', 'partyCity': 'PHOENIX', 'partyZip1': '85004', 'partyZip2': '1252', 'partyState': 'AZ', 'partyPhone': '(602)496-8888', 'nielsenDma': 'PHOENIX (PRESCOTT)', 'networkAfil': 'PBS', 'band': 'High V', 'authAppId': '1316399'}, {'id': '7143', 'callSig

In [17]:
# Create folders for each station.
def createStationFolders(s):
    path = "/home/andrew/Documents/PredictiveModeling/TV/stations/"+s
    try: 
        os.mkdir(path)
    except FileExistsError:
        print(path+" exists")
    try:
        os.mkdir(path+"/Political Files")
    except FileExistsError:
        print('')
    try:
        os.mkdir(path+"/Political Files/2018")
    except FileExistsError:
        print('')
tv['callSign'].apply(createStationFolders)

/home/andrew/Documents/PredictiveModeling/TV/stations/KAET exists


0     None
1     None
4     None
5     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
Name: callSign, dtype: object

In [5]:
#Get the head political Folder
cycle = '18'
def getFileFolder(iD):
    url = 'https://publicfiles.fcc.gov/api/manager/folder/path.json'
    folder = 'Political Files/20'+cycle
    r = requests.get(url, params={"folderPath":folder, "entityId": iD, 'sourceService':'tv'}).json()
    if r['statusCode'] == 200:
        return r['folder'][0]['entity_folder_id']
    else:
        return None
tv[(cycle+'PoliticalFolder')] = tv['id'].apply(getFileFolder)

In [19]:
# Create all directory folders
cycle = '18'
import time
def createBuyFolders(s):
    print(s)
    if s[(cycle+'PoliticalFolder')] != None:
        url = "https://publicfiles.fcc.gov/api/manager/folder/id/"+s[(cycle+'PoliticalFolder')]+".json"
        digToNextLevel(s[(cycle+'PoliticalFolder')], s['id'], s['callSign'])
def digToNextLevel(fId, tvId, sCs):
    path = "/home/andrew/Documents/PredictiveModeling/TV/stations/"
    url = "https://publicfiles.fcc.gov/api/manager/folder/id/"+fId+".json"
    r = requests.get(url,params={'entityId':tvId}).json()
    if r['statusCode'] == 200:
        try:
            os.mkdir(path+sCs+"/"+r['folder']['folder_path'])
        except FileExistsError:
            print(path+sCs+"/"+r['folder']['folder_path']+" exists")
        if len(r['folder']['subfolders']) > 0:
            for sf in r['folder']['subfolders']:
                digToNextLevel(sf["entity_folder_id"], sf["entity_id"], sCs)
    else:
        print(r['statusMessage'])
        time.sleep(10)
        digToNextLevel(fId, tvId, sCs)
tv[[(cycle+'PoliticalFolder'),'id','callSign']].apply(createBuyFolders, axis=1)

18PoliticalFolder    a195b284-3823-9311-3fd0-ab1eb7945dce
id                                                   2728
callSign                                             KAET
Name: 0, dtype: object
/home/andrew/Documents/PredictiveModeling/TV/stations/KAET/Political Files/2018 exists
18PoliticalFolder    3dd55d88-d62c-0cff-eeb8-822f84cdc06e
id                                                   7143
callSign                                             KASW
Name: 1, dtype: object
/home/andrew/Documents/PredictiveModeling/TV/stations/KASW/Political Files/2018 exists
18PoliticalFolder    f532836f-2248-72e2-1974-cca90b96998d
id                                                  59440
callSign                                          KNXV-TV
Name: 4, dtype: object
/home/andrew/Documents/PredictiveModeling/TV/stations/KNXV-TV/Political Files/2018 exists
18PoliticalFolder    872c413b-96b9-549e-e6bc-1710e893d9bf
id                                                  67868
callSign                     

0     None
1     None
4     None
5     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
dtype: object

In [108]:
stations = [];
for f in list(os.scandir("/media/andrew/F08C9B848C9B444E/analysis/tv/buys/")):
    f.is_dir()

In [2]:
pacs = pd.read_csv('pacs.csv')

In [242]:
juris = ['Federal', 'Local', 'Non-Candidate Issue Ads', 'State']
fedrace = ['President', 'US Senate', 'US House']
pacs = []
def getAllPacs(s):
    basepath = "/home/andrew/Documents/PredictiveModeling/TV/stations/"+s['sign']+'/Political Files/'+cycle+'/'
    try:
        os.scandir(basepath)
        for j in juris:
            path = basepath + j
            if j == 'Federal':
                for r in fedrace:
                    fedpath = path+'/'+r
                    if r == 'US House': # Sometimes House folders have districts
                        dist = []
                        fcontents = list(os.scandir(fedpath)) 
                        if len(fcontents) >0:
                            for f in fcontents:## US House
                                if f.is_dir():
                                    distpath = fedpath+'/'+f.name
                                    fileFound = False
                                    for p in list(os.scandir(distpath)): ## Subfolder, is it district or pac?
                                        if p.is_dir() == True:
                                            pacs.append({"Name": p.name, "Jurisdiction": j, "Race": None, "Station":s['sign'], 
                                                 "Path": f.path, 'District': f.name, "Cycle": cycle})
                                        else:
                                            fileFound=True
                                    if (fileFound) or len(list(os.scandir(distpath))) == 0:
                                        pacs.append({"Name": f.name, "Jurisdiction": j, "Race": None, "Station":s['sign'], 
                                                 "Path": f.path, 'District': None, "Cycle": cycle})   
                        else:
                            for f in fcontents:
                                pacs.append({"Name": p.name, "Jurisdiction": j, "Race": None, "Station":s['sign'], "Path": p.path, 'District': None, "Cycle": cycle})
                    else:
                        for p in list(os.scandir(fedpath)):
                            if p.is_dir():
                                pacs.append({"Name": p.name, "Jurisdiction": j, "Race": None, "Station":s['sign'], "Path": p.path, 'District': None, "Cycle": cycle})
            if j == 'State':
                fcontents = list(os.scandir(path))
                if len(fcontents)>0:
                    for r in fcontents: # Loop through folders in the State folder.
                        # Folder two contents,
                        fileFound = False
                        f2path = path+'/'+r.name
                        f2contents = list(os.scandir(f2path))
                        for f in f2contents:
                            if (f.is_dir()) & ("Invoices" in f.name == False):
                                pacs.append({"Name": f.name, "Jurisdiction": j, "Race": r.name, 
                                             "Station":s['sign'], "Path": f.path,'District': None,
                                             "Cycle": cycle})
                            else:
                                fileFound = True
                        if (fileFound) or (len(f2contents)==0):
                            pacs.append({"Name": r.name, "Jurisdiction": j, "Race": None, "Station":s['sign'], 
                                        "Path": r.path, 'District': None, "Cycle": cycle})
            if (j != 'Federal') & (j != 'State'):
                for p in list(os.scandir(path)):
                    if p.is_dir():
                        pacs.append({"Name": p.name, "Jurisdiction": j, "Race": None, "Station":s['sign'], "Path": p.path, 'District': None, "Cycle": cycle})
    except:
        return

cycles = ['2018'] #,'2016','2014']

for c in cycles:
    cycle = c
    tv.apply(getAllPacs, axis=1)
pacs = pd.DataFrame(pacs)
pacs.to_csv('pacs.csv', index=False)

In [292]:
p = pd.read_csv('pacs.csv')
p.head()

,Cycle,District,Jurisdiction,Name,Path,Race,Station
0,2018,NaN,Federal,Kyrsten Sinema,/home/andrew/Documents/PredictiveModeling/TV/s...,NaN,KASW
1,2018,NaN,Federal,Kyrsten Sinema DSCC,/home/andrew/Documents/PredictiveModeling/TV/s...,NaN,KASW
2,2018,NaN,Federal,McSally for Senate,/home/andrew/Documents/PredictiveModeling/TV/s...,NaN,KASW
3,2018,AZ 09,Federal,Greg Stanton for Congress,/home/andrew/Documents/PredictiveModeling/TV/s...,NaN,KASW
4,2018,AZ 01,Federal,DCCC T OHALLERAN AZ-1,/home/andrew/Documents/PredictiveModeling/TV/s...,NaN,KASW


In [293]:
#
# Preprocess folder names to make it easier to resolve them to their other committees.
#
# Get info from name.
p['NameSimpl'] = p['Name']
p['Party'] = None

# Transform words
p['NameSimpl'] = p['NameSimpl'].str.replace('AZDP',' DEMOCRAT ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace('AZRP',' REPUBLICAN', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace('AZDEMS',' DEMOCRATIC PARTY ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace('AZ ','ARIZONA ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' AZ',' ARIZONA', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' Prty ',' Party ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' ATT ',' ATTORNEY ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' ATY ',' ATTORNEY ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' SEC ',' SECRETARY ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' GEN ',' GENERAL ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' SOS ',' SECRETARY OF STATE ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' SOS',' SECRETARY OF STATE ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' AG ',' ATTORNEY GENERAL ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' AG',' ATTORNEY GENERAL', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' DEM ',' DEMOCRATIC ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' REP ',' REPUBLICAN ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' DEMS ',' DEMOCRATIC PARTY ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' GOV ', ' GOVERNOR ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' PHX ', ' PHOENIX ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace('National Republican Senatorial Committee','NRSC', case=False)

# IE
p.loc[p['Jurisdiction'] == 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace(' IE', '', case=False)

#PAC
p.loc[p['Jurisdiction'] == 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace(' 4 ', ' For ', case=False)
p.loc[p['Jurisdiction'] == 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace(' ASSC ', ' Association ', case=False)
p.loc[p['Jurisdiction'] == 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('auth', '', case=False)
p.loc[p['Jurisdiction'] == 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('ACLU', 'American Civil Liberties Union', case=False)

#Candidate
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('For', '', case=False)
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 
      'NameSimpl'] = p['NameSimpl'].str.replace('- General Election', '', case=False)
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 
      'NameSimpl'] = p['NameSimpl'].str.replace('- Primary Election', '', case=False)
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 
      'NameSimpl'] = p['NameSimpl'].str.replace('Phoenix', '', case=False)
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 
      'NameSimpl'] = p['NameSimpl'].str.replace('Arizona', '', case=False)

#Party
p.loc[p['Name'].str.contains(' R ', case=False), 'Party'] = "Republican"
p.loc[p['Name'].str.contains(' D ', case=False), 'Party'] = "Democrat"
p.loc[p['NameSimpl'].str.contains('Republican', case=False), 'Party'] = "Republican"
p.loc[p['NameSimpl'].str.contains('Democrat', case=False), 'Party'] = "Democrat"
p['NameSimpl'] = p['NameSimpl'].str.replace(' D ','', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' R ','', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace('Democratice', 'Democratic')

#Federal
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace(' US ', '', case=False)
#Senate
p.loc[p['Name'].str.contains('Senate', case=False), 'Race'] = "Senate"
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('Senate', '', case=False)
p.loc[p['Name'].str.contains('DSCC', case=False), 'Race'] = "Senate"
p.loc[p['Name'].str.contains('DSCC', case=False), 'Party'] = "Democrat"
p.loc[p['Name'].str.contains('NRSC', case=False), 'Race'] = "Senate"
p.loc[p['Name'].str.contains('NRSC', case=False), 'Party'] = "Republican"
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('DSCC', '', case=False)
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('NRSC', '', case=False)

# House
p.loc[p['Name'].str.contains('House', case=False), 'Race'] = "House"
p.loc[p['Name'].str.contains('Congress', case=False), 'Race'] = "House"
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('House', '', case=False)
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('Congress', '', case=False)
p.loc[p['Name'].str.contains('DCCC', case=False), 'Race'] = "House"
p.loc[p['Name'].str.contains('DCCC', case=False), 'Party'] = "Democrat"
p.loc[p['Name'].str.contains('NRCC', case=False), 'Race'] = "House"
p.loc[p['Name'].str.contains('NRCC', case=False), 'Party'] = "Republican"
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('NRCC', '', case=False)
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('DCCC', '', case=False)

#State
p.loc[p['NameSimpl'].str.contains('Arizona Democratic Party', case=False), 'Party'] = "Democrat"
p.loc[p['NameSimpl'].str.contains('Arizona Republican Party', case=False), 'Party'] = "Republican"
#Governor
p.loc[p['NameSimpl'].str.contains('RGA', case=False), 'Party'] = "Republican"
p.loc[p['NameSimpl'].str.contains('DGA', case=False), 'Race'] = "Governor"
p.loc[p['NameSimpl'].str.contains('RGA', case=False), 'Race'] = "Governor"
p.loc[p['NameSimpl'].str.contains('Governor', case=False), 'Race'] = "Governor"
p.loc[p['NameSimpl'].str.contains('DGA', case=False), 'Party'] = "Democrat"
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('Governor', '', case=False)
#AG
p.loc[p['NameSimpl'].str.contains('Attorney General', case=False), 'Race'] = "Attorney General"
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('Attorney General', '', case=False)
p.loc[p['NameSimpl'].str.contains('RAGA', case=False), 'Party'] = "Republican"
p.loc[p['NameSimpl'].str.contains('DAGA', case=False), 'Race'] = "Attorney General"
p.loc[p['NameSimpl'].str.contains('RAGA', case=False), 'Race'] = "Attorney General"
p.loc[p['NameSimpl'].str.contains('DAGA', case=False), 'Party'] = "Democrat"
p.loc[p['Jurisdiction'] == 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('DAGA', ' Democratic Attorneys General Association ', case=False)


#SOS
p.loc[p['NameSimpl'].str.contains('Secretary of State', case=False), 'Race'] = "Secretary of State"
p['NameSimpl'] = p['NameSimpl'].str.replace('Secretary of State', '', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace('Secretary State', '', case=False)
#Superintendent
p.loc[p['NameSimpl'].str.contains('Superintendent', case=False), 'Race'] = "Superintendent of Public Instruction"
p['NameSimpl'] = p['NameSimpl'].str.replace('Superintendent of Public Education', ' ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace('State Superintendent', '', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace('Superintendent', '', case=False)
# State Treasurer
p.loc[p['NameSimpl'].str.contains('Treasurer', case=False), 'Race'] = "State Treasurer"
p['NameSimpl'] = p['NameSimpl'].str.replace('State Treasurer', ' ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace('Treasurer', '', case=False)
#Mayor#
p.loc[p['NameSimpl'].str.contains('Mayor', case=False), 'Race'] = "Mayor"
p['NameSimpl'] = p['NameSimpl'].str.replace('Mayor', '', case=False)

# General vs Primary
p.loc[(p['Name'].str.contains('ATTORNEY', case=False) == False) & 
      (p['Name'].str.contains('GENERAL', case=False)), 'NameSimpl'] = p['NameSimpl'].str.replace('GENERAL', '', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace('PRIMARY','', case=False)

#All
p['NameSimpl'] = p['NameSimpl'].str.replace('-',' ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace('2018', '')
p['NameSimpl'] = p['NameSimpl'].str.replace('2016', '')
p['NameSimpl'] = p['NameSimpl'].str.replace('2014', '')

#Party Committees
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('Democratic Party', '', case=False)
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('Republican Party', '', case=False)
p.loc[(p['Jurisdiction'] != 'Non-Candidate Issue Ads'),
      'NameSimpl'] = p['NameSimpl'].str.replace('Republican', '', case=False)
p.loc[(p['Jurisdiction'] != 'Non-Candidate Issue Ads'),
      'NameSimpl'] = p['NameSimpl'].str.replace('Democrat', '', case=False)
p.loc[(p['Jurisdiction'] != 'Non-Candidate Issue Ads'),
      'NameSimpl'] = p['NameSimpl'].str.replace('Party', '', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace('Behalf of', '', case=False)
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('auth', '', case=False)

# Put everything that's local in state because stations can't agree on what's what.
p.loc[p['Jurisdiction']=='Local', 'Jurisdiction'] = "State"

# Acronym Expansion
p.loc[p['Jurisdiction'] == 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('PFAW', 'People for the American Way', case=False)
p.loc[p['Jurisdiction'] == 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('RGA', 'Republican Governors Association', case=False)
p.loc[p['Jurisdiction'] == 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('DCCC', 'Democratic Congressional Campaign Committee', case=False)


p['Cycle'] = p['Cycle'].astype(str)
p = p.replace({np.nan:None})
p['NameSimpl'] = p['NameSimpl'].str.strip()
p[p['NameSimpl'].str.contains(' DEMOCRAT   January Contreas  ')].head()

,Cycle,District,Jurisdiction,Name,Path,Race,Station,NameSimpl,Party


In [43]:
## DSCC and DCCC Expanded still above 70%, solution: limit NCIA to same race and side

fuzz.token_sort_ratio('Democratic Attorneys General Association - Democratic Attorneys General Association 2018','Democratic Attorneys General Association Peoples Lawyers Project')

59

In [294]:
matched = []
curName = ""
committeeCount=0
uniqueCom = pd.DataFrame(columns=['Cycle', 'District', 'Jurisdiction', 
                                  'Name', 'Path', 'Race', 'Station', 'Party',
                                 'NameSimpl'])
dedupedCom = pd.DataFrame(columns=['Cycle', 'District', 'Jurisdiction', 
                                  'Name', 'Path', 'Race', 'Station', 'Party',
                                 'NameSimpl', 'MatchedName', 'MatchedStation'])
# Use fuzzy string matching to link the same committees accross stations.
def matcher(r):
    global uniqueCom
    global dedupedCom
    matcher = {'Race': None, 'District': None, 'Party': None}
    def closestNames(n):
        nonlocal matcher
        matchVal = fuzz.token_sort_ratio(n.NameSimpl, r.NameSimpl)
        matchVal2 = fuzz.token_set_ratio(n.NameSimpl, r.NameSimpl)
        if r['Jurisdiction'] == 'Non-Candidate Issue Ads':
            matchThreshold = 75
        else:
            matchThreshold = 78
        # Potentially average sort and set for matchVal rate.
        if (matchVal+matchVal2)/2 > matchThreshold:
            if (n.Race is not None) & (matcher['Race'] is None):
                matcher['Race'] = n.Race
            if (n.District is not None) & (matcher['District'] is None):
                matcher['District'] = n.District
            if (n.Party is not None) & (matcher['Party'] is None):
                matcher['Party'] = n.Party
            return True
        else:
            return False
    def fixDupComs(match, replace):
        global uniqueCom
        global dedupedCom
        print("Fixing duplicate committees...")
        print(match['NameSimpl'].values)
        print(r['NameSimpl'])
        # Replace already matched committees with better committee name.
        dedupedCom.loc[dedupedCom['MatchedName'].isin(match['Name'].values), 'MatchedName'] = replace['Name']
        badindexes = uniqueCom[uniqueCom['Name'].isin(match['Name'].values)].index.values
        uniqueCom = uniqueCom.drop(badindexes)
    possmatch = uniqueCom[(uniqueCom.Cycle==r.Cycle)&(uniqueCom.Jurisdiction==r.Jurisdiction)]
    match = possmatch[possmatch.apply(closestNames, axis=1)]
    # Check already matched committees.
    if(len(match) == 0):
        possmatch = dedupedCom[(dedupedCom.Cycle==r.Cycle)&(dedupedCom.Jurisdiction==r.Jurisdiction)]
        match = possmatch[possmatch.apply(closestNames, axis=1)]
        if len(match)>0:
            match['Name'] = match['MatchedName']
            match['Station'] = match['MatchedStation']
            match = uniqueCom[uniqueCom['Name'].isin(match[0:1]['MatchedName'].values)][0:1]
    if len(match) == 1:
        if (matcher['Race'] is None) & (r['Race'] is not None):
            uniqueCom.loc[match.index.values, 'Race'] = r['Race']
        if (matcher['District'] is None) & (r['District'] is not None):
            uniqueCom.loc[match.index.values, 'District'] = r['District']
        if (matcher['Party'] is None) & (r['Party'] is not None):
            uniqueCom.loc[match.index.values, 'Party'] = r['Party']
        r['MatchedName'] = match[0:1]['Name'].values[0]
        r['MatchedStation'] = match[0:1]['Station'].values[0]
    else:
        if(len(match['Name'].unique())>1):
            print(match['Name'].unique())
            fixDupComs(match, r)
        r['MatchedName'] = r['Name']
        r['MatchedStation'] =r['Station']
        uniqueCom = uniqueCom.append(r)
    dedupedCom = dedupedCom.append(r)
    return r
pz = p.apply(matcher, axis=1)
uniqueCom.head()
# Error types: Duplicate committees, false matches.

['Vote Vets Action Fund' 'Majority Forward']
Fixing duplicate committees...
['Vote Vets Action Fund' 'Majority Forward']
MAJORITY FORWARD VOTE VETS ACTION FUND


/home/andrew/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/andrew/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


['Ferrara for Congress' 'STEVE FERRARA - US CONGRESS AS-09']
Fixing duplicate committees...
['Ferrara' 'STEVE FERRARA   AS 09']
STEVE FERRARA    09
['TIPRNENI' 'HIRAL TIPIRNENI DCONGRESS']
Fixing duplicate committees...
['TIPRNENI' 'HIRAL TIPIRNENI D']
HIRAL TIPRNENI D


,Cycle,District,Jurisdiction,Name,Path,Race,Station,Party,NameSimpl,MatchedName,MatchedStation
0,2018,None,Federal,Kyrsten Sinema,/home/andrew/Documents/PredictiveModeling/TV/s...,Senate,KASW,Democrat,Kyrsten Sinema,Kyrsten Sinema,KASW
2,2018,None,Federal,McSally for Senate,/home/andrew/Documents/PredictiveModeling/TV/s...,Senate,KASW,Republican,McSally,McSally for Senate,KASW
3,2018,AZ 09,Federal,Greg Stanton for Congress,/home/andrew/Documents/PredictiveModeling/TV/s...,House,KASW,Democrat,Greg Stanton,Greg Stanton for Congress,KASW
4,2018,AZ 01,Federal,DCCC T OHALLERAN AZ-1,/home/andrew/Documents/PredictiveModeling/TV/s...,House,KASW,Democrat,T OHALLERAN 1,DCCC T OHALLERAN AZ-1,KASW
7,2018,None,Non-Candidate Issue Ads,Tax March,/home/andrew/Documents/PredictiveModeling/TV/s...,None,KASW,None,Tax March,Tax March,KASW


In [335]:
# Merge in deduped committee primary keys, then write committees key to disk.
coms18 = uniqueCom[uniqueCom['Cycle']=='2018'].sort_values(by='NameSimpl').reset_index(drop=True)
coms18['id'] = coms18.index
coms18['MatchedName'] = coms18['Name']
dedupedCom2=dedupedCom[dedupedCom['Cycle']=="2018"][1:].merge(coms18[['id', 'MatchedName']], 
                                                              how='left', on=['MatchedName'])
dedupedCom2.to_csv('adcommitteekey.csv', index=False)
coms18.head()
#dedupedCom2[dedupedCom2['Race'].isna()]

,Cycle,District,Jurisdiction,Name,Path,Race,Station,Party,NameSimpl,MatchedName,MatchedStation,id
0,2018,None,State,Governor,/home/andrew/Documents/PredictiveModeling/TV/s...,Governor,KASW,None,,Governor,KASW,0
1,2018,None,Non-Candidate Issue Ads,45 Committe PAC,/home/andrew/Documents/PredictiveModeling/TV/s...,None,KASW,None,45 Committe PAC,45 Committe PAC,KASW,1
2,2018,None,Non-Candidate Issue Ads,AFSCME People Pac,/home/andrew/Documents/PredictiveModeling/TV/s...,None,KASW,None,AFSCME People Pac,AFSCME People Pac,KASW,2
3,2018,None,Non-Candidate Issue Ads,ARIZONA EDUCATION PROJECT,/home/andrew/Documents/PredictiveModeling/TV/s...,None,KPHO-TV,None,ARIZONA EDUCATION PROJECT,ARIZONA EDUCATION PROJECT,KPHO-TV,3
4,2018,None,Non-Candidate Issue Ads,ACLU AZ,/home/andrew/Documents/PredictiveModeling/TV/s...,None,KNXV-TV,None,American Civil Liberties Union ARIZONA,ACLU AZ,KNXV-TV,4


In [171]:

de# Various duplication tests.

#uniqueCom[uniqueCom['NameSimpl'].str.contains('Gallego', case=False)]
#dedupedCom[dedupedCom['Cycle']=="2018"]['MatchedName'].unique()
#p[p['Name'].str.contains('January')]['Name'].unique()
#dedupedCom[(dedupedCom['MatchedName'].str.contains('January')) | (dedupedCom['MatchedName'].str.contains('Contre'))]['MatchedName'].unique()
#dedupedCom[dedupedCom['NameSimpl'].str.contains('Congressional')][0:10]

NameError: name 'de' is not defined

In [336]:
# Merge in hand coded IE spends and sides.
iers = pd.read_csv('ies.csv')
ies = coms18[(coms18['Jurisdiction'] == 'Non-Candidate Issue Ads') & (coms18['Cycle']=='2018')]
ies = ies[['Name', 'MatchedName', 'Station', 'id', 'NameSimpl']].merge(iers.drop('NameSimpl', axis=1), how="left", on=['Name', 'Station'])
uniqueComs2 = coms18[(coms18['Jurisdiction'] != 'Non-Candidate Issue Ads') & 
                       (coms18['Cycle'] == '2018')].append(ies)
uniqueComs2.to_csv('uniqueComPreFinal.csv')
## WRITE THIS WITH A UNIQUE NAME AND TO A CERTAIN FOLDER
uniqueComs2[(uniqueComs2['Race'].isna()) | (uniqueComs2['Party'].isna())].to_csv('comsNeedingInfo.csv')
#uniqueComs2[(uniqueComs2['Race'].isna()) | (uniqueComs2['Party'].isna())]
#uniqueComs2.head()
# Write to CSV todo manual final edits

/home/andrew/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [341]:
# Write final unique committees to csv for database upload
finalComs = pd.read_csv('uniqueComPreFinal.csv')
comsWithInfo = pd.read_csv('comsNeedingInfo1012.csv')

finalComs = finalComs[(finalComs['Race'].isna() == False) & (finalComs['Party'].isna() == False)]
finalComs = finalComs.append(comsWithInfo)
finalComs = finalComs.drop(['Unnamed: 0', 'id'], axis=1)
finalComs = finalComs.sort_values(by='NameSimpl', na_position='first').reset_index(drop=True)
finalComs['id'] = finalComs.index
finalComs.columns = map(str.lower, finalComs.columns)
finalComs = finalComs.drop(['matchedname', 'name', 
                'station', 'path', 'matchedstation'], axis=1)
finalComs.rename(columns={'namesimpl':'name',
                            'party':'side',
                            'jurisdiction':'juris'}).to_csv('comsupload.csv', index=False)
#finalComs[(finalComs['race'].isna()) | (finalComs['party'].isna())][['name', 'race', 'party']]
finalComs.head()

/home/andrew/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,cycle,district,jurisdiction,namesimpl,party,race,id
0,2018,NaN,State,NaN,Democrat,Governor,0
1,2018,NaN,Non-Candidate Issue Ads,45 Committe PAC,Republican,President,1
2,2018,NaN,Non-Candidate Issue Ads,AFSCME People Pac,Democrat,Senate,2
3,2018,NaN,Non-Candidate Issue Ads,ARIZONA EDUCATION PROJECT,Corporate Tax Breaks,Red for Ed,3
4,2018,NaN,Non-Candidate Issue Ads,American Civil Liberties Union ARIZONA,Democrat,Senate,4


In [379]:
key = pd.read_csv('adcommitteekey.csv')
ads = pd.read_csv('scrapedads.csv')
stations = pd.read_csv('stationsPhoenix.csv')
# Merge in commitees
key = key.rename(columns={'Name':'Committee', 'id':'pacid'})
# Running on empty directories fails the pac search for AZ09, pac search would pick this up on your desktop.
ads.loc[ads.Committee == 'AZ 09', 'Committee'] = 'Greg Stanton for Congress'
ads = ads.merge(key[['Committee', 'pacid']], on='Committee', how='left') .drop('Committee', axis=1)
ads['pacid'] = ads['pacid'].astype(int)
# Merge in stations
ads = ads.rename(columns={'Station':'sign'})
stations = stations.rename(columns={'id':'stationid'})
ads = ads.merge(stations[['stationid', 'sign']], on="sign") .drop(['sign'], axis=1)
# Map columns to database schema, write to file.
ads = ads.rename(columns={'Date':'airdate'})
ads.columns = map(str.lower, ads.columns)
# Format rate properly because I was too stupid to do it in the scraping code.
#ads['rate'] = ads['rate'].str[1:-2].str.replace(',','').astype(int)
#ads.to_csv('adsupload.csv', index=False)
ads['rate'] = ads['rate'].fillna('0')
ads['rate'] = ads['rate'].str.replace('$','', regex=False)
ads['rate'] = ads['rate'].str.replace(',','', regex=False).astype(f;l)
ads['rate'] = ads['rate']/100
ads[ads['rate']].head()
#ads.info()

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [251]:
ads[ads.pacid.isna()][['Committee', 'sign']].groupby(by=['Committee','sign']).sum()
#key[key.pacid.isna()]

,
Committee,sign
AZ 09,KASW


In [276]:
pacs[pacs['Station'] == 'KASW']

,Cycle,District,Jurisdiction,Name,Path,Race,Station
0,2018,None,Federal,Kyrsten Sinema,/home/andrew/Documents/PredictiveModeling/TV/s...,None,KASW
1,2018,None,Federal,Kyrsten Sinema DSCC,/home/andrew/Documents/PredictiveModeling/TV/s...,None,KASW
2,2018,None,Federal,McSally for Senate,/home/andrew/Documents/PredictiveModeling/TV/s...,None,KASW
3,2018,AZ 09,Federal,Greg Stanton for Congress,/home/andrew/Documents/PredictiveModeling/TV/s...,None,KASW
4,2018,AZ 01,Federal,DCCC T OHALLERAN AZ-1,/home/andrew/Documents/PredictiveModeling/TV/s...,None,KASW
5,2018,AZ 01,Federal,OHALLERAN AZ-1,/home/andrew/Documents/PredictiveModeling/TV/s...,None,KASW
6,2018,None,Federal,TIPRNENI,/home/andrew/Documents/PredictiveModeling/TV/s...,None,KASW
7,2018,None,Non-Candidate Issue Ads,Tax March,/home/andrew/Documents/PredictiveModeling/TV/s...,None,KASW
8,2018,None,Non-Candidate Issue Ads,Arizonans for Affordable Electricity,/home/andrew/Documents/PredictiveModeling/TV/s...,None,KASW
9,2018,None,Non-Candidate Issue Ads,AFSCME People Pac,/home/andrew/Documents/PredictiveModeling/TV/s...,None,KASW


,airdate,program,rate,cycle,pacid,stationid
